In [1]:
import pandas as pd
# import re
from datetime import datetime
import string

In [2]:
# Correção para linhas que começam com múltiplos espaços

def correctSplit(x):
    splitted_str = list(x)
    if splitted_str[0] == '|':
        splitted_str[0] = ''
    return ''.join(splitted_str)

# Função para separar colunas do df importado

def separar_colunas_df(df):
    # lista alfabetica e.g ['a','b','c'...]
    lista_alfabeto = list(string.ascii_lowercase)
    # Pega a primeira coluna do df (que tem todos os dados) e substitui onde possui mais de um espaço por | 
    df.iloc[:,0] = df.iloc[:,0].replace({'( ){2,} ':'|'},regex=True)
    # corrige a substituição para linhas que começam com espaços
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x: correctSplit(x))
    # corta a partir do separador '|' e define cada pedaço para uma coluna ['a','b'..]
    for i in range (len(df.iloc[:,0][0].split('|'))):
        df[lista_alfabeto[i]] = df.iloc[:,0].apply(lambda x: x.split('|')[i])
    # deleta a coluna que originou as demais colunas    
    df.drop(df.columns[0], axis = 1, inplace = True)
    return df

# veic_2019_02.csv_amostra

In [3]:
colnames = ['index_df', 'x', 'y', 'data-hora', 'data', 'dia', 'mes', 
            'ano', 'tempo', 'hora', 'a',
           'b', 'c', 'd', 'e', 'f', 'placa', 'h', 
            'i', 'j', 'k', 'l' ]

df_veic = pd.read_csv('veic_2019_02.csv_amostra.txt', sep=",", names=colnames)
df_veic.head(4)

,index_df,x,y,data-hora,data,dia,mes,ano,tempo,hora,...,c,d,e,f,placa,h,i,j,k,l
0,index,n,x,tipo,data,dia,mes,ano,hora,y,...,vel2,k,s,cod,placa,r,t,y,u,i
1,1,501,1,2019-02-01 00:00:07,2019-02-01,01,02,2019,00:00:07,00,...,68.0,2,3.8,bdac468f1,LRV5999,0,0,NaN,NaN,NaN
2,2,502,2,2019-02-01 00:00:09,2019-02-01,01,02,2019,00:00:09,00,...,57.0,2,3.8,bdac469a2,LRW4982,0,0,NaN,NaN,NaN
3,3,502,2,2019-02-01 00:00:17,2019-02-01,01,02,2019,00:00:17,00,...,61.0,2,3.6,bdac46ee2,LQM9654,0,0,NaN,NaN,NaN


# movimento_OCR_JAN_2019.txt_amostra.txt

In [5]:
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_ocr_raw = pd.read_csv('movimento_OCR_JAN_2019.txt_amostra.txt', sep='|', names = colnames)

df_ocr_split = separar_colunas_df(df_ocr_raw.copy())
df_ocr_split.head()

,a,b,c,d,e
0,KPV68771AUTOMÓVEL,20190101000000971024,0101201900000003900CET8-5455 0410191112025201...,St R Catete,B-22.93380-43.18294
1,LQR82471AUTOMÓVEL,20190101000001181044,0101201900000103400CET8-5410 0410121122025201...,St Tanque,D-22.97019-43.41377
2,ETP18901AUTOMÓVEL,20190101000001281014,0101201900000104000CET8-5436 0410241111025201...,St Cidade Nova,A-22.93149-43.20888
3,KXL99161AUTOMÓVEL,20190101000001381014,0101201900000102500CET8-5439 0410161111025201...,St Colonia,A-22.92479-43.38907
4,LPU22871AUTOMÓVEL,20190101000001901024,0101201900000104400CET8-5401 0410381122025201...,St Pontal,B-23.01267-43.38900


# Tratando OCR

In [7]:
def get_datahora(string):
    date = string[38:52]
    try: 
        date = datetime.strptime(date, "%Y%m%d%H%M%S")
    except:
        date = 'erro'
    return date

def get_velocidade(string):
    vel = string[92:95]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

def get_placa(string):
    placa = string[0:7]
    return placa.strip()

df_ocr = pd.DataFrame()
df_ocr['CODCET'] = df_ocr_raw['x'].apply(lambda x: x[108:118])
df_ocr['DATAHORA'] = df_ocr_raw['x'].apply(lambda x: get_datahora(x))
df_ocr['PLACA'] = df_ocr_raw['x'].apply(lambda x: get_placa(x))
df_ocr['VELOCIDADE'] = df_ocr_raw['x'].apply(lambda x: get_velocidade(x))
df_ocr['CGT'] = df_ocr_raw['x'].apply(lambda x: x[108:111])
df_ocr['LATITUDE'] = df_ocr_raw['x'].apply(lambda x: x[216:225])
df_ocr['LONGITUDE'] = df_ocr_raw['x'].apply(lambda x: x[225:234])

# Salvando em csv
df_ocr.to_csv('amostra_movimento_tratada.csv', sep=';', index=False)
df_ocr.sample(5)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGT,LATITUDE,LONGITUDE
29,0410552121,2019-01-01 00:00:10,KRR1791,31,041,-23.00726,-43.43819
54,0410191111,2019-01-01 00:00:21,LRP8425,29,041,-22.93380,-43.18294
4,0410381122,2019-01-01 00:00:01,LPU2287,44,041,-23.01267,-43.38900
42,0410351111,2019-01-01 00:00:17,KXK4498,29,041,-23.01567,-43.41202
73,0410551111,2019-01-01 00:00:31,JEO4197,27,041,-23.00850,-43.43940


# SCP_INTVIA_0182018_201901_amostra.txt

In [8]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_intvia_raw = pd.read_csv('SCP_INTVIA_0182018_201901_amostra.txt', sep='|', names = colnames)

# separando as colunas
df_intvia_split = separar_colunas_df(df_intvia_raw)

df_intvia_split.head()

,a,b,c,d,e,f
0,-------1moto,0101201900000215,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
1,LSY84841moto,0101201900000417,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"2-22,96635-43,21941"
2,-------1moto,0101201900000714,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
3,-------1moto,0101201900002229,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
4,-------1moto,0101201900004443,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"


# SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt

In [9]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_sitran_raw = pd.read_csv('SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas
df_sitran_split = separar_colunas_df(df_sitran_raw)

df_sitran_split.head()

,a,b,c,d,e,f
0,-------1moto,0103201910263053,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"1-22,90393-43,27340"
1,-------1moto,0103201910263141,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"2-22,90393-43,27340"
2,-------1moto,0103201910263449,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"3-22,90393-43,27340"
3,LQS56581moto,0103201910490948,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"1-22,90393-43,27340"
4,FLT20441moto,0103201910491046,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"3-22,90393-43,27340"


# SCP_TRANSCARIOCA_0222016_201903_amostra.txt

In [12]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_transcarioca_raw = pd.read_csv('SCP_TRANSCARIOCA_0222016_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas
df_transcarioca_split = separar_colunas_df(df_transcarioca_raw.copy())
df_transcarioca_split.head(5)

,a,b,c,d,e,f
0,-------2carro de passeio,010320190010319,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
1,-------1moto,0103201900104424,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
2,-------2carro de passeio,0103201900104419,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
3,-------1moto,0103201900104628,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
4,-------1moto,0103201900104830,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"


# Tratamento SCP_TRANSCARIOCA

In [25]:
def get_datahora(string):
    date = string[78:92]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date

def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_velocidade(string):
    vel = string[92:94]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

def get_latitude(string):
    lat = string[216:219]+"."+string[220:225]
    return lat

def get_longitude(string):
    lon = string[225:228]+"."+string[229:]
    return lon

df_transcarioca = pd.DataFrame()
df_transcarioca['CODCET'] = df_transcarioca_raw['x'].apply(lambda x:x[108:118])
df_transcarioca['DATAHORA'] = df_transcarioca_raw['x'].apply(lambda x: get_datahora(x))
df_transcarioca['PLACA'] = df_transcarioca_raw['x'].apply(lambda x: get_placa(x))
df_transcarioca['VELOCIDADE'] = df_transcarioca_raw['x'].apply(lambda x: get_velocidade(x))
df_transcarioca['CGCT'] = df_transcarioca_raw['x'].apply(lambda x:x[108:111])
df_transcarioca['LATITUDE'] = df_transcarioca_raw['x'].apply(lambda x: get_latitude(x))
df_transcarioca['LONGITUDE'] = df_transcarioca_raw['x'].apply(lambda x: get_longitude(x))

# Salvando em csv
df_transcarioca.to_csv('amostra_transcarioca_tratada.csv', sep=';', index=False)
df_transcarioca.head(20)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGCT,LATITUDE,LONGITUDE
0,0440021120,2019-03-01 00:10:31,,9,044,-22.87280,-43.33629
1,0440021120,2019-03-01 00:10:44,,24,044,-22.87280,-43.33629
2,0440021120,2019-03-01 00:10:44,,19,044,-22.87280,-43.33629
3,0440021120,2019-03-01 00:10:46,,28,044,-22.87280,-43.33629
4,0440021120,2019-03-01 00:10:48,,30,044,-22.87280,-43.33629
5,0440021120,2019-03-01 00:10:50,,30,044,-22.87280,-43.33629
6,0440021120,2019-03-01 00:10:58,,34,044,-22.87280,-43.33629
7,0440021120,2019-03-01 00:11:07,,32,044,-22.87280,-43.33629
8,0440021120,2019-03-01 00:11:08,,33,044,-22.87280,-43.33629
9,0440021120,2019-03-01 00:11:11,,34,044,-22.87280,-43.33629


# SPLICE_007201820190205.txt_amostra.txt

In [22]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df = pd.read_csv('SPLICE_0072018_20190205_.txt_amostra.txt', sep='|', names = colnames)
# df_splice_raw
# Separando as colunas manualmente

# df_splice_split = separar_colunas_df(df_splice_raw)

# def separar_colunas_df(df):
lista_alfabeto = list(string.ascii_lowercase)

df.iloc[:,0] = df.iloc[:,0].replace({'( ){2,} ':'|'},regex=True)

lista_e = []
df.iloc[:,0] = df.iloc[:,0].apply(lambda x: correctSplit(x))
for i in range(len(df)):
    lista_e.append(len(df.iloc[:,0][i].split('|'))==4)
#     print(len(df.iloc[:,0][i].split('|')))

# df.iloc[3][0][78:]

a = df.x.where(lista_e)
print(a.dropna().iloc[0])
print(len(a.dropna().iloc[0]))
print(df.iloc[0][0])
print(len(df.iloc[0][0]))
# print(df.shape)
# print(len(lista_e))
# print(df.iloc[:,0][0].split('|'))
# for i in range (len(df.iloc[:,0][0].split('|'))):
#     try:
#         df[lista_alfabeto[i]] = df.iloc[:,0].apply(lambda x: x.split('|')[i])
#     except:
#         df[lista_alfabeto[i]] = 'erro'
# df.drop(df.columns[0], axis = 1, inplace = True)

# df

# df_splice['x'] = df_splice.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
# df_splice['x'] = df_splice.x.apply(lambda x: correctSplit(x))
# df_splice['veiculo'] = df_splice.x.apply(lambda x: x.split('|')[0])
# df_splice['a'] = df_splice.x.apply(lambda x: x.split('|')[1])
# df_splice['b'] = df_splice.x.apply(lambda x: x.split('|')[2])
# df_splice['c'] = df_splice.x.apply(lambda x: x.split('|')[3])

# Dropando a coluna x utilizada para separar as outras colunas
# df_splice.drop('x', axis = 1, inplace = True)

# df_splice_split.sample(5)

1MOTO|2019010100004705300RD3399|04505612120072018AV PRES ANTONIO CARLOS PX ED MENEZES CORTES PLAT-ST CANDELARCandelaria|2-22.90751-43.17307
139
1MOTO|2019010100000105900RD3389|04503811120072018AV GAL SAN MARTIN PX 220|Sao Conrado|2-22.98468-43.21814
105


In [23]:
i = 0
print(df.iloc[i][0][78:], end = '\n\n')
print(df.iloc[i][0][30:], end = '\n\n')
print(df.iloc[i][0][140:], end = '\n\n')

Conrado|2-22.98468-43.21814

9|04503811120072018AV GAL SAN MARTIN PX 220|Sao Conrado|2-22.98468-43.21814





# FISCALTECH_0042016_06_2019.txt_amostra.txt

In [24]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_fiscaltech = pd.read_csv('FISCALTECH_0042016_06_2019.txt_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_fiscaltech['x'] = df_fiscaltech.x.replace({'( ){3,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_fiscaltech['x'] = df_fiscaltech.x.apply(lambda x: correctSplit(x))
df_fiscaltech['a'] = df_fiscaltech.x.apply(lambda x: x.split('|')[0])
df_fiscaltech['b0'] = df_fiscaltech.x.apply(lambda x: x.split('|')[1])
df_fiscaltech['b'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[0])
df_fiscaltech['c'] = df_fiscaltech.b0.apply(lambda x: ' '.join(list(x.split(' ')[1:])))
df_fiscaltech['d'] = df_fiscaltech.x.apply(lambda x: x.split('|')[2])
# df_fiscaltech['c'] = df_fiscaltech.x.apply(lambda x: x.split('|')[3])

# Dropando a coluna x utilizada para separar as outras colunas
df_fiscaltech.drop(['x','b0'], axis = 1, inplace = True)

df_fiscaltech.sample(5)

,a,b,c,d
21,1,0106201903332904300FSCII2202,04201411220042016AvS.Allende BRT OPalme-PCent...,2-22981046-43409698
66,2,0106201903301805200FSCII2199,04201311220042016AvAbelardoBueno Px980 PCent-...,2-22973217-43387464
2,1,0106201901120105000FSCII2174,04200711240042016Av Americas Px2151 PCent-St ...,4-23001615-43328645
78,1,0106201901130105600FSCII2196,04201811120042016AvAbelardoBueno Px3500 PCent...,2-22973304 -4339867
88,1,0106201903324001400FSCII2175,04200611140042016Av Americas Px2000 PCent-St ...,4-23000409-43334161
